In [1]:
import numpy as np
import pandas as pd

In [2]:
# importing the data

df1 = pd.read_csv('battery_station_1.csv')
df2 = pd.read_csv('battery_station_2.csv')
df3 = pd.read_csv('intermediate/服務中心.csv')
df4 = pd.read_csv('intermediate/others.csv')

In [3]:
# cleaning the columns of battery station data

df = pd.concat([df1, df2], ignore_index=True)
df = df.drop(['description', 'timestamp', 'begin', 'end', 'altitudeMode', 'tessellate', 'extrude', 'visibility', 'drawOrder', 'icon', 'NameOnPage', 'Address'], axis=1)
df = df.rename(columns={'______':'address', '____________': 'status', 'Latitude': 'lat', 'Longitude': 'lng'})

In [4]:
def get_activate_deactivate_time(records):
    activate_time = []
    deactivate_time = []
    for record in eval(records):
        if record[-4:] == '~ 啟用':
            activate_time.append(record[:-5])
        if record[-11:] == '~ 將已啟用改回建置中':
            deactivate_time.append(record[:-12])

    return activate_time, deactivate_time

In [5]:
# generating multiple columns for multiple activate and deactivate times

activate_times, deactivate_times = zip(*df['Records'].apply(get_activate_deactivate_time))

df_result = df.copy()

for update_times in (activate_times, deactivate_times):

    maxlen = max([len(time) for time in update_times]) # getting the maximum length of activate times

    # generating column names with number equal to the maximum number of activation times
    new_cols = []
    if update_times == activate_times:
        col_pre = 'activate_time'
    elif update_times == deactivate_times:
        col_pre = 'deactivate_time'

    for i in range(1, maxlen+1):
        new_cols.append(col_pre + str(i))

    # generating a dictionary with keys as column names defined above and filling the values in
    dict = {}
    # iterate through number of activate times
    for i in range(maxlen):
        column_list = []
        for time in update_times:
            # the station does not have as many times of update as i
            if i >= len(time):
                column_list.append(np.nan)
            else:
                column_list.append(time[i])

        dict[new_cols[i]] = column_list

    # dictionary to dataframe
    df_update = pd.DataFrame.from_dict(dict)

    # merging back to the original dataframe
    df_result = pd.concat([df_result, df_update], axis=1)

# saving the data
df_result.to_csv('results/battery_stations.csv', index=False)

In [6]:
# cleaning the service center data

df3 = df3.drop(['description', 'timestamp', 'begin', 'end', 'altitudeMode', 'tessellate', 'extrude', 'visibility', 'drawOrder'], axis=1)
df3.to_csv('results/service_centers.csv', index=False)

In [7]:
# cleaning the others data

df4['lat'] = df4['description'].apply(lambda des: des[des.index('緯度')+4:des.index('<br>經度')])
df4 = df4.rename(columns = {'______': 'lng', '____________': 'status'})
df4 = df4[['Name', 'status', 'lat', 'lng']]
df4.to_csv('results/others.csv')